In [5]:
import pandas as pd
data = pd.read_csv("C:/Users/82107/Desktop/동아리/프로젝트/final_data.CSV").dropna()x

,Unnamed: 0,momentum,value,size,quality,risk,return,date
0,A005930,0.932561,-0.899528,1.730061,1.647266,-0.813157,0.002639,2001-07-31 00:00:00
4,A005490,0.749455,-0.293724,1.719447,0.166880,1.208820,0.026077,2001-07-31 00:00:00
5,A005380,0.485443,0.110146,1.711487,0.279927,0.133707,-0.034014,2001-07-31 00:00:00
6,A000270,-1.017726,-0.444257,1.706180,0.312227,0.791327,-0.016166,2001-07-31 00:00:00
10,A006400,0.515251,-0.157876,1.692912,1.571901,0.103691,0.000000,2001-07-31 00:00:00
...,...,...,...,...,...,...,...,...
921725,A008560,1.281020,1.510267,1.610484,-0.371407,0.266699,0.068493,2022-11-30 00:00:00
921768,A010050,0.722557,0.075887,1.265018,-0.399795,1.303614,-0.053435,2022-11-30 00:00:00
922338,A048260,1.137777,-1.454688,1.495813,-0.005520,1.110873,0.241906,2022-11-30 00:00:00
922353,A050540,-1.705306,1.657767,-1.722253,0.544888,0.670110,0.000000,2022-11-30 00:00:00


In [4]:
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# 데이터 불러오기 (이 부분은 데이터를 불러오는 방식에 따라 다를 수 있습니다)
# data = pd.read_csv('your_data.csv')

# 롤링 윈도우 방식으로 데이터를 학습하고 다음 달의 주가 예측
rolling_window = 5  # 5개월
predicted_returns = {}  # 사전으로 초기화

for stock_id in tqdm(data['stock_id'].unique()):
    stock_data = data[data['stock_id'] == stock_id]
    stock_predicted_returns = []

    for end_month in range(rolling_window, len(stock_data)):
        # 학습 데이터와 테스트 데이터 분리
        train_data = stock_data.iloc[end_month - rolling_window:end_month]
        test_data = stock_data.iloc[end_month:end_month + 1]

        # 데이터 특징 및 타겟 분리
        X_train = train_data[['momentum', 'value', 'size', 'quality', 'risk']]
        y_train = train_data['return']
        X_test = test_data[['momentum', 'value', 'size', 'quality', 'risk']]

        # 데이터 표준화
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # SVR 모델 학습 및 예측
        svr_model = SVR()
        svr_model.fit(X_train_scaled, y_train)
        predicted_return = svr_model.predict(X_test_scaled)

        # 예측된 수익률 저장
        stock_predicted_returns.extend(predicted_return)

    predicted_returns[stock_id] = stock_predicted_returns

# 데이터에 예측된 수익률 추가
for stock_id, returns in predicted_returns.items():
    data.loc[data['stock_id'] == stock_id, 'predicted_return'] = pd.Series(returns)

# 결과 확인
data.head()

# 나머지 전략 적용 코드는 이전 코드와 동일하게 사용


KeyError: 'stock_id'